## 5.1 层和块

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)

net(X)

tensor([[ 0.2746,  0.3412, -0.1586,  0.0041, -0.0107,  0.3586, -0.0521,  0.0193,
         -0.1387, -0.1226],
        [ 0.3598,  0.1883, -0.2044,  0.0004,  0.0427,  0.3626, -0.1189,  0.0550,
         -0.1193, -0.1034]], grad_fn=<AddmmBackward0>)

### 5.1.1 自定义块

In [4]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)
  
  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))


In [5]:
net = MLP()
net(X)

tensor([[-0.0843,  0.2058,  0.2834, -0.0033,  0.0602, -0.0123,  0.1696, -0.2561,
         -0.0686,  0.3246],
        [-0.0524,  0.1676,  0.2481, -0.0669,  0.1110, -0.0534, -0.0144, -0.2073,
         -0.0859,  0.3758]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序快

In [6]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self._modules[str(idx)] = module


  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2007, -0.2004,  0.2691, -0.0549,  0.1563,  0.1220,  0.0730, -0.1350,
         -0.1251,  0.0282],
        [ 0.1650, -0.1214,  0.2482, -0.1143,  0.1645,  0.0794,  0.0606, -0.2818,
         -0.0836,  0.0846]], grad_fn=<AddmmBackward0>)

### 5.1.3 在前向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum(0)

In [15]:
net = FixedHiddenMLP()
net(X)

tensor([ 0.0655, -0.0216, -0.0316, -0.0350,  0.0532, -0.0492,  0.1080,  0.0104,
         0.0175,  0.0419,  0.0433,  0.0069, -0.0242, -0.0723, -0.0152, -0.0176,
        -0.0330,  0.0271,  0.0314,  0.0235], grad_fn=<SumBackward1>)

In [16]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                             nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)
  
  def forward(self, X):
    return self.linear(self.net(X))

In [17]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor([ 0.0340,  0.0290,  0.0231,  0.0500, -0.0071,  0.0160, -0.0217,  0.0210,
        -0.0187, -0.0058, -0.0111, -0.0554, -0.0167, -0.0302, -0.0227,  0.0135,
         0.0284, -0.0244,  0.0759, -0.0155], grad_fn=<SumBackward1>)

In [19]:
# 2. 实现平行块


class MyNet1(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(20, 256),
      nn.ReLU(),
      nn.Linear(256, 40)
    )
  
  def forward(self, X):
    return self.net(X)

class MyNet2(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(40, 20),
      nn.ReLU(),
      nn.Linear(20, 10)
    )
  
  def forward(self, X):
    return self.net(X)
  
mynet = nn.Sequential(MyNet1(), MyNet2())
mynet(X)

tensor([[ 0.2088, -0.0857,  0.3436, -0.0044,  0.2110,  0.0057,  0.1934, -0.0516,
          0.0959, -0.2448],
        [ 0.2062, -0.0670,  0.3424, -0.0007,  0.2077, -0.0176,  0.1722, -0.0668,
          0.0930, -0.2480]], grad_fn=<AddmmBackward0>)

## 5.2 参数管理

In [20]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1317],
        [0.1959]], grad_fn=<AddmmBackward0>)

### 5.2.1 参数访问

In [21]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2192, -0.3289,  0.1030,  0.0718,  0.1117,  0.1602, -0.0326,  0.3160]])), ('bias', tensor([0.2024]))])


In [22]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2024], requires_grad=True)
tensor([0.2024])


In [23]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [24]:
net.state_dict()['2.bias'].data

tensor([0.2024])

In [25]:
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
         nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1707],
        [0.1705]], grad_fn=<AddmmBackward0>)

In [26]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [27]:
rgnet[0][1][0].bias.data

tensor([-0.1695,  0.4668,  0.1065,  0.1952, -0.2350,  0.2330,  0.0304, -0.2151])

### 5.2.2 参数初始化

In [29]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0080, -0.0002, -0.0106, -0.0092]), tensor(0.))

In [31]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)

net.apply(init_constant)

net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [32]:
def init_xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].apply(init_xavier))
print(net[2].apply(init_42))

Linear(in_features=4, out_features=8, bias=True)
Linear(in_features=8, out_features=1, bias=True)


In [33]:
def my_init(m):
  if type(m) == nn.Linear:
    print("Init", *[(name, param.shape) 
                    for name, param in m.named_parameters()][0])
    
    nn.init.uniform_(m.weight, -10, 10)
    m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)

net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.5857, -0.0000,  0.0000,  8.0813],
        [-8.0285, -0.0000, -7.4692, -6.2517]], grad_fn=<SliceBackward0>)

In [34]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  9.0813])

### 5.2.3 参数绑定

In [36]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)

print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 5.3 延后初始化

## 5.4 自定义层

### 5.4.1 不带参数的层

In [37]:
import torch
import torch.nn.functional as F
from torch import nn

In [40]:
class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X):
    return X - X.mean()

In [41]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [42]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [43]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.7789e-09, grad_fn=<MeanBackward0>)

### 5.4.2 带参数的层

In [44]:
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.randn(units,))
  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)


In [45]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.3761, -1.4402,  0.3196],
        [ 0.1689,  1.4698,  0.7660],
        [ 0.2874,  0.2391, -0.7497],
        [-0.2998,  0.0715, -1.4424],
        [-1.0007, -2.6984,  1.1358]], requires_grad=True)

In [46]:
linear(torch.rand(2, 5))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [47]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))

net(torch.rand(2, 64))

tensor([[0.0000],
        [0.6949]])

## 5.5 读写文件

### 5.5.1 加载和保存张量

In [48]:
import torch
from torch import nn
from torch.nn import functional as F

In [49]:
x = torch.arange(4)

torch.save(x, 'x-file')

In [51]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [52]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')

(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [53]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')

mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.5.2 加载和保存模型参数

In [54]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.output = nn.Linear(256, 10)

  def forward(self, x):
    return self.output(F.relu(self.hidden(x)))

In [55]:
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [56]:
torch.save(net.state_dict(), 'mlp.params')

In [57]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [58]:
Y_clone = clone(X)
Y_clone = Y

## 5.6 GPU

In [59]:
!nvidia-smi

Sun Apr 28 14:13:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.61                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050       WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   39C    P8               N/A /  N/A|    131MiB /  3072MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 5.6.1 计算设备

In [60]:
import torch
from torch import nn

In [61]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [62]:
torch.cuda.device_count()

0

In [63]:
def try_gpu(i=0): #@save
  if torch.cuda.device_count() >= i + 1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus(): #@save
  devices = [torch.device(f'cuda:{i}')
              for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

### 5.6.2 张量与GPU

In [64]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [65]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [66]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.0970, 0.6463, 0.8881],
        [0.4972, 0.8371, 0.7720]])

In [68]:
X = X.cuda(1)
print(X)
print(Z)

AssertionError: Torch not compiled with CUDA enabled

In [69]:
Y + Z

NameError: name 'Z' is not defined

In [70]:
Z.cuda(1) is Z

NameError: name 'Z' is not defined

### 5.6.3 神经网络与GPU

In [ ]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [ ]:
net(X)

In [ ]:
net[0].weight.data.device